In [14]:
import sys
import copy
import numpy as np

In [15]:
def read_file(text_path):
    _tmp = []
    with open(text_path, 'r', encoding="utf-8") as f:
        for line in f:
            line = line.split()
            _tmp.append(line)
    return np.array(_tmp).astype(float)

In [25]:
def find_relation_of_atom(lines):
    atom_relation_dic = {}
    beginnings = {}

    # calculate the distance, 
    # and fit the atom index whoes distance between 3.7 and 3.9
    for i in range(len(lines)):
        _tmp = []
        for j in range(len(lines)):
            _distance = np.sqrt(np.sum( (lines[i][1:]-lines[j][1:])**2 ))
            if (_distance<3.9) and (_distance>3.7):
                _tmp.append(lines[j][0])
        if len(_tmp) == 1:
            beginnings[lines[i][0]] = _tmp
        if len(_tmp) > 0:
            atom_relation_dic[lines[i][0]] = _tmp

    return atom_relation_dic, beginnings


In [17]:
def list2dic(lines):
    atom_dic = {}
    for line in lines:
        atom_dic[int(line[0])] = line[1:]
    return atom_dic

In [54]:
def cal_center_distance(atom_dic, atom_relation_dic):
    center_distance = {}
    for index in atom_relation_dic.keys():
        _tmp = [0,0,0]
        _tmp_len = len(atom_relation_dic[index])
        _tmp_distance = []

        for relate_atom in atom_relation_dic[index]:
            _tmp += atom_dic[relate_atom]
        center_posi = np.array(_tmp)/_tmp_len

        for relate_atom in atom_relation_dic[index]:
            _distance = np.sqrt(np.sum( ( atom_dic[relate_atom] - center_posi )**2 ))
            _tmp_distance.append(_distance)
        
        center_distance[index] = _tmp_distance
        
        # closest_atom[index] = atom_relation_dic[index][np.array(_tmp_distance).argmin()]
    return center_distance

In [55]:
def find_most_relation_atom(center_distance, atom_relation_dic):
    most_relation_atom = {}

    for index in center_distance.keys():
        _tmp = []

        min_distance = np.min(np.array(center_distance[index]))
        for _inx, _dist in enumerate(center_distance[index]):
            if abs(_dist-min_distance)<1.2:
                _tmp.append(int(atom_relation_dic[index][_inx]))

        most_relation_atom[int(index)] = _tmp
    
    return most_relation_atom


In [19]:
text_path = './data_q2.txt'
lines = read_file(text_path)
atom_dic = list2dic(lines)
atom_relation_dic, beginnings = find_relation_of_atom(lines)
# all_center_distance = cal_center_distance(atom_dic, atom_relation_dic)
# most_relation_atom = find_most_relation_atom(all_center_distance, atom_relation_dic)

In [57]:
def find_path(child, most_relation_atom,chain,chains):
    # explored[child] = most_relation_atom[child]
    _tmp_related_atom = most_relation_atom[child]
    chain = copy.deepcopy(chain)
    chain.append(child)

    for _child in _tmp_related_atom:
        if _child in chain:
            _tmp_related_atom.remove(_child)
        
        if _child not in list(most_relation_atom.keys()):
            _tmp_related_atom.remove(_child)
    
    if len(_tmp_related_atom) == 0:
        chains.append(chain)
        return

    for child in _tmp_related_atom:
        find_path(child, most_relation_atom,chain,chains)

In [20]:
import math
def cal_angle(point_a, point_b, point_c):

    a_x, b_x, c_x = point_a[0], point_b[0], point_c[0]
    a_y, b_y, c_y = point_a[1], point_b[1], point_c[1]
    a_z, b_z, c_z = point_a[2], point_b[2], point_c[2]

    x1,y1,z1 = (a_x-b_x),(a_y-b_y),(a_z-b_z)
    x2,y2,z2 = (c_x-b_x),(c_y-b_y),(c_z-b_z)


    cos_b = (x1*x2 + y1*y2 + z1*z2) / (math.sqrt(x1**2 + y1**2 + z1**2) *(math.sqrt(x2**2 + y2**2 + z2**2)))
    B = math.degrees(math.acos(cos_b))
    return B

cal_angle((1,1,1), (0,0,0), (1,0,1))

35.26438968275466

In [31]:
def find_path_2(root, child1, atom_relation_dic, chain, chains, atom_dic):
    _tmp_related_atom1 = atom_relation_dic[child1]

    chain = copy.deepcopy(chain)
    chain.append(child1)

    for _child in _tmp_related_atom1:
        if _child in chain:
            _tmp_related_atom1.remove(_child)
        
        if _child not in list(atom_relation_dic.keys()):
            _tmp_related_atom1.remove(_child)
    
    if len(_tmp_related_atom1) == 0:
        chains.append(chain)
        print(chain)
        return

    for child2 in _tmp_related_atom1:
        _angle = cal_angle(atom_dic[root],atom_dic[child1],atom_dic[child2])
        if (75 < _angle < 160) or (-105 < _angle < -20):
            root = child1
            child1 = child2
            find_path_2(root, child1, atom_relation_dic, chain, chains, atom_dic)
        

In [37]:
text_path = './data_q2.txt'
lines = read_file(text_path)
atom_dic = list2dic(lines)
atom_relation_dic, beginnings = find_relation_of_atom(lines)

chains = []

for child1 in beginnings.keys():
    chain = []
    chain.append(child1)

    for child2 in atom_relation_dic[child1]:
        if (child2 not in chain) and (child2 in atom_relation_dic.keys()):
            find_path_2(child1, child2, atom_relation_dic, chain, chains, atom_dic)

len_chain = []
for chain in chains:
    len_chain.append(len(chain))

np.max(len_chain)

[25.0, 31.0]
[35.0, 19.0, 4.0, 3.0, 16.0, 27.0, 54.0, 72.0, 70.0, 86.0, 142.0, 169.0, 158.0, 207.0, 242.0, 215.0, 202.0, 143.0, 163.0, 110.0, 152.0, 165.0, 104.0, 119.0, 174.0, 221.0, 211.0, 139.0]
[35.0, 19.0, 4.0, 3.0, 16.0, 27.0, 54.0, 72.0, 70.0, 86.0, 142.0, 169.0, 158.0, 207.0, 242.0, 215.0, 202.0, 143.0, 163.0, 110.0, 152.0, 165.0, 104.0, 119.0, 174.0, 231.0, 229.0, 168.0, 94.0]
[37.0, 49.0, 66.0, 64.0]
[46.0, 81.0]
[55.0, 38.0, 65.0, 59.0, 36.0, 29.0]
[75.0, 66.0, 64.0]
[87.0, 135.0, 134.0, 79.0, 71.0]
[89.0, 97.0]
[92.0, 158.0, 207.0, 242.0, 215.0, 202.0, 143.0, 163.0, 110.0, 152.0, 165.0, 104.0, 119.0, 174.0, 221.0, 211.0, 139.0]
[92.0, 158.0, 207.0, 242.0, 215.0, 202.0, 143.0, 163.0, 110.0, 152.0, 165.0, 104.0, 119.0, 174.0, 231.0, 229.0, 168.0, 94.0]
[93.0, 148.0, 174.0, 231.0, 229.0, 168.0, 94.0]
[98.0, 115.0, 154.0, 176.0, 122.0, 145.0]
[100.0, 99.0, 161.0, 179.0, 167.0, 118.0, 128.0]
[101.0, 135.0, 134.0, 79.0, 71.0]
[103.0, 144.0, 217.0, 189.0, 205.0, 223.0, 202.0, 143.

40

In [93]:
chains = []
for start in beginnings.keys():
    find_path(start, most_relation_atom,[],chains)
len_chain = []

for chain in chains:
    len_chain.append(len(chain))

for atom in chains[np.argmax(len_chain)]:
    print(atom)
print("The total number of alpha-carbon atoms in the chain {}".format(np.max(len_chain)))

20.0
9
32
15
44
62
63
69
83
117
160
177
203
237
256
266
301
294
243
236
227
205
223
273
262
215
181
116
106
184
207
129
158
169
142
86
70
72
54
27
16
3
4
The total number of alpha-carbon atoms in the chain 43


43

In [84]:
chains = []
for start in beginnings.keys():
    find_path(start, most_relation_atom,[],chains)
len_chain = []

for chain in chains:
    len_chain.append(len(chain))

np.max(len_chain)

43

In [71]:
for chain in chains:
    len_chain.append(len(chain))

np.max(len_chain)

48

In [ ]:
def find_most_relation_atom(center_distance, atom_relation_dic):
    most_relation_atom = {}

    for index in center_distance.keys():
        _tmp = []

        min_distance = np.min(np.array(center_distance[index]))
        for _inx, _dist in enumerate(center_distance[index]):
            if abs(_dist-min_distance)<1.2:
                _tmp.append(int(atom_relation_dic[index][_inx]))

        most_relation_atom[int(index)] = _tmp
    
    return most_relation_atom

In [ ]:
def dfs(chain, child, atom_relation_dic):
    chain = copy.deepcopy(chain)
    
    if child not in chain:
        chain.append(child) 
        print(chain)

        if child not in list(atom_relation_dic.keys()):
            return

        elif set(atom_relation_dic[child]) < set(chain):
            return

        else:
            root = child
            for child in atom_relation_dic[root]:
                dfs(chain, child, most_relation_atom)
    else:return